## Spatial statistics and generalised interpolation

Notebook investigating multidimensional interpolation, including Radial Basis Functions (RBF) and Kriging examples

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

### Linear weighting

Consider the real function $f(X)$ where $X$ is a vector in an m-dimensional space. Take also a set of points $X_i$ with $i$ running from $1$ to $N$. We can form the vector $F$ with $F_i = f(X_i)$.

Define the estimated function $g(X) = \sum_{i=1}^N W_i(X) F_i = W(X).F$, where $W(X)$ is a vector of weights; in general these weights are a function of $X$. One-dimensional linear interpolation, for example, corresponds to the case where $m=1$ and the weights are determined by distances to the nearest neighbour(s).

Two cases of interest are RBF and Kriging. The general approach is to define the function $f(X)$, then a procedure for preparing the values needed to pass to a weight generating function that determines $W(X)$; in other words precalculation of the parameters used within $W(X)$ is desirable for efficiency. 

### RBF

Radial basis functions are based around a kernel function $k(A,B)$ that represents some idea of the distance between points $A$ and $B$. In general the kernel function is symmetric, so $k(A,B) = k(B,A)$, and also translationally invariant, so $k(A,B) = \phi(A-B)$.

Consider the function $h(X) = \sum_i k(X,X_i) \alpha_i$ for a vector $\alpha$ of factors, one for each point $X_i$. Then we can form $H$ according to $H_i = h(X_i)$, and then find the factors $\alpha$ that minimize the regularized
likelihood $L = -\frac{1}{2}\left(F-H\right)^2 + \frac{\lambda}{2}\left(\alpha K \alpha\right)$, where the matrix $K$ has been formed as $K_{ij} = k(X_i,X_j)$, so $H = K.\alpha$.

Minimizing the likelihood yields $\alpha = \left(K-\lambda\right)^{-1} F$. Note that this means that the estimated function is of the linear weighting form described above.